# Blackjack Simulation
I recently watched a video, [Link](https://www.youtube.com/watch?v=7nBigBH1r04&ab_channel=Crumb) detailing a blackjack betting operation from MIT students. Obviously it's enticing. Maths = Money, stealing from the rich. But while money is to be made, there are always people watching, stopping you. Especially casinos. Anyways, to 'see what it's like', so to speak, I wanted to build a simulator, which plays blackjack using the MIT strategy. 

But, the MIT strategy only improves odds of winning by 1%? The casino still has the edge... how do they make money. Anyways, moral of the story is, I hate gambling and this should hopefully prove that you shouldn't gamble.

References:



# Imports

In [2]:
import random
import pandas as pd

In [22]:
# --- Constants ---
suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
ranks = {
    '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7,
    '8': 8, '9': 9, '10': 10, 'J': 10, 'Q': 10, 'K': 10, 'A': 11
}

true_count_zones = {
    "Highly Unfavourable": lambda tc: tc <= 0,
    "Unfavourable": lambda tc: tc == 1,
    "Neutral": lambda tc: tc == 2,
    "Favourable": lambda tc: tc == 3,
    "Highly Favourable": lambda tc: tc >= 4
}

# Deviation overrides: (player total, dealer upcard) -> (threshold, new action)
index_deviations = {
    (16, 10): (0, 'S'),
    (15, 10): (4, 'S'),
    (10, 10): (4, 'D'),
    (12, 3): (2, 'S'),
    (12, 2): (3, 'S'),
    (13, 2): (-1, 'S'),
    (13, 3): (-2, 'S'),
    (12, 4): (0, 'S'),
    (12, 5): (-2, 'S'),
    (12, 6): (-1, 'S'),
    (11, 11): (1, 'D'),
    (10, 11): (4, 'D'),
    (10, 10): (4, 'D'),
    (10, 9): (0, 'D'),
    (9, 2): (1, 'D'),
    (9, 7): (3, 'D'),
    (8, 6): (2, 'D'),
    (8, 5): (3, 'D'),
    (8, 4): (4, 'D'),
    (13, 10): (9, 'S'),
    (14, 10): (3, 'S'),
    (15, 9): (2, 'S'),
    (15, 11): (5, 'S'),
    (16, 9): (5, 'S'),
    (16, 8): (10, 'S')
}

# Strategy tables
hard_totals = {
    5:  ['H'] * 10, 6: ['H'] * 10, 7: ['H'] * 10, 8: ['H'] * 10,
    9:  ['H','D','D','D','D','H','H','H','H','H'],
    10: ['D']*8 + ['H','H'],
    11: ['D']*9 + ['H'],
    12: ['H','H','S','S','S','H','H','H','H','H'],
    13: ['S']*5 + ['H']*5,
    14: ['S']*5 + ['H']*5,
    15: ['S']*5 + ['H']*4 + ['SR','H'],
    16: ['S']*5 + ['H']*2 + ['SR','SR','SR'],
    17: ['S'] * 10, 18: ['S'] * 10, 19: ['S'] * 10,
    20: ['S'] * 10, 21: ['S'] * 10
}
soft_totals = {
    13: ['H','H','H','D','D','H','H','H','H','H'],
    14: ['H','H','H','D','D','H','H','H','H','H'],
    15: ['H','H','D','D','D','H','H','H','H','H'],
    16: ['H','H','D','D','D','H','H','H','H','H'],
    17: ['H','D','D','D','D','H','H','H','H','H'],
    18: ['S','D','D','D','D','S','S','H','H','H'],
    19: ['S'] * 10, 20: ['S'] * 10, 21: ['S'] * 10
}
pairs = {
    4:  ['P','P','P','P','P','P','H','H','H','H'],
    6:  ['P','P','P','P','P','P','H','H','H','H'],
    8:  ['H','H','H','P','P','H','H','H','H','H'],
    10: ['P','P','P','P','P','H','H','H','H','H'],
    12: ['P','P','P','P','P','P','H','H','H','H'],
    14: ['P','P','P','P','P','P','H','H','H','H'],
    16: ['P'] * 10,
    18: ['P','P','P','P','P','S','P','P','S','S'],
    20: ['S'] * 10,
    22: ['P'] * 10
}

class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.value = ranks[rank]

class Deck:
    def __init__(self, num_decks=6, penetration=0.75):
        self.num_decks = num_decks
        self.penetration = penetration
        self.full_deck = [Card(suit, rank) for suit in suits for rank in ranks] * num_decks
        self.shuffle()

    def shuffle(self):
        self.cards = self.full_deck.copy()
        random.shuffle(self.cards)
        self.running_count = 0

    def deal(self):
        reshuffle_point = int((1 - self.penetration) * len(self.full_deck))
        if len(self.cards) <= reshuffle_point:
            self.shuffle()
        card = self.cards.pop()
        self.update_count(card)
        return card

    def update_count(self, card):
        if card.rank in ['10', 'J', 'Q', 'K', 'A']:
            self.running_count -= 1
        elif card.rank in ['2', '3', '4', '5', '6']:
            self.running_count += 1

    def true_count(self):
        decks_remaining = len(self.cards) / 52
        return round(self.running_count / decks_remaining) if decks_remaining > 0 else 0

# --- Helpers ---
def hand_value(hand):
    value = sum(card.value for card in hand)
    aces = sum(1 for card in hand if card.rank == 'A')
    while value > 21 and aces:
        value -= 10
        aces -= 1
    return value

def is_pair(hand):
    return len(hand) == 2 and hand[0].rank == hand[1].rank

def is_soft_hand(hand):
    has_ace = any(card.rank == 'A' for card in hand)
    return has_ace and hand_value(hand) != sum(card.value for card in hand)

def get_upcard_index(card):
    if card.rank in ['10', 'J', 'Q', 'K']: return 8
    if card.rank == 'A': return 9
    return int(card.rank) - 2

def get_bet_from_true_count(tc):
    if tc <= 0: return 1
    elif tc == 1: return 2
    elif tc == 2: return 4
    elif tc == 3: return 6
    else: return 8

def get_player_action(hand, dealer_card, true_count=0, mode="strategy"):
    value = hand_value(hand)
    upcard_index = get_upcard_index(dealer_card)
    dealer_val = dealer_card.value if dealer_card.rank != 'A' else 11

    if mode == "basic":
        return 'H' if value < 17 else 'S'

    if mode == "index" and not is_soft_hand(hand) and not is_pair(hand):
        key = (value, dealer_val)
        if key in index_deviations:
            threshold, alt_action = index_deviations[key]
            if true_count >= threshold:
                return alt_action

    if is_pair(hand):
        pair_total = hand[0].value * 2
        return pairs.get(pair_total, ['H']*10)[upcard_index]
    elif is_soft_hand(hand):
        return soft_totals.get(value, ['H']*10)[upcard_index]
    else:
        return hard_totals.get(value, ['H']*10)[upcard_index]

def play_hand_with_strategy(deck, mode="strategy"):
    player = [deck.deal(), deck.deal()]
    dealer = [deck.deal(), deck.deal()]
    dealer_upcard = dealer[0]
    true_count = deck.true_count()
    doubled = False

    while True:
        action = get_player_action(player, dealer_upcard, true_count, mode=mode)
        if action == 'H':
            player.append(deck.deal())
            if hand_value(player) > 21:
                break
        elif action == 'D':
            if len(player) == 2:
                player.append(deck.deal())
                doubled = True
            break
        elif action == 'S':
            break
        elif action == 'P':
            player.append(deck.deal())
            break
        elif action == 'SR':
            return "Loss", False

    while hand_value(dealer) < 17:
        dealer.append(deck.deal())

    pv, dv = hand_value(player), hand_value(dealer)
    if pv > 21: return "Loss", doubled
    elif dv > 21: return "Win", doubled
    elif pv > dv: return "Win", doubled
    elif pv < dv: return "Loss", doubled
    return "Draw", doubled

def simulate_blackjack_compare_modes(num_decks=6, num_hands=5000, penetration=0.75):
    modes = {
        "Basic": "basic",
        "MIT Strategy": "strategy",
        "MIT + Index": "index"
    }

    summaries = []

    for label, mode in modes.items():
        deck = Deck(num_decks=num_decks, penetration=penetration)
        bankroll = 0
        results = []

        for _ in range(num_hands):
            true_count = deck.true_count()
            base_bet = get_bet_from_true_count(true_count)
            outcome, doubled = play_hand_with_strategy(deck, mode=mode)
            actual_bet = base_bet * 2 if doubled else base_bet

            if outcome == "Win":
                bankroll += actual_bet
            elif outcome == "Loss":
                bankroll -= actual_bet

            results.append({
                "Mode": label,
                "Outcome": outcome,
                "Bet": actual_bet,
                "Profit": actual_bet if outcome == "Win" else (-actual_bet if outcome == "Loss" else 0),
                "Bankroll": bankroll
            })

        df = pd.DataFrame(results)
        win_rate = (df["Outcome"] == "Win").mean()
        summaries.append({
            "Mode": label,
            "Hands Played": num_hands,
            "Final Bankroll": bankroll,
            "Win Rate %": round(win_rate * 100, 2),
            "Total Bet": df["Bet"].sum(),
            "ROI %": round((bankroll / df["Bet"].sum()) * 100, 2)
        })

    return pd.DataFrame(summaries)


In [23]:
summary_df = simulate_blackjack_compare_modes(num_decks=6, num_hands=1000000)
print(summary_df)

           Mode  Hands Played  Final Bankroll  Win Rate %  Total Bet  ROI %
0         Basic       1000000         -165739       40.82    2120520  -7.82
1  MIT Strategy       1000000         -169262       40.66    2256757  -7.50
2   MIT + Index       1000000         -142354       41.18    2330795  -6.11


In [15]:
import numpy as np

# Simulation parameters
starting_money = 500
target_money = 1_000_000
num_simulations = 100_000
results = {"reach_million": 0, "go_broke": 0}

# Run simulations
for _ in range(num_simulations):
    money = starting_money
    while money > 0 and money < target_money:
        bet = money
        if np.random.rand() < 0.5:
            money += bet  # win
        else:
            money -= bet  # lose

    if money >= target_money:
        results["reach_million"] += 1
    else:
        results["go_broke"] += 1

results

{'reach_million': 45, 'go_broke': 99955}

In [20]:
import numpy as np
import plotly.graph_objects as go

# Parameters
starting_money = 500
num_simulations = 100_000
max_money_list = []

# Simulation loop
for _ in range(num_simulations):
    money = starting_money
    max_money = money
    while money > 0:
        bet = money
        if np.random.rand() < 0.5:
            money += bet
        else:
            money -= bet
        max_money = max(max_money, money)
    max_money_list.append(max_money)

# Create histogram using Plotly
fig = go.Figure(data=[go.Histogram(x=max_money_list, nbinsx=100)])
fig.update_layout(
    title="Maximum Money Reached Before Going Broke (100,000 Simulations)",
    xaxis_title="Max Money Before Loss",
    yaxis_title="Frequency (log scale)",
    yaxis_type="log",
    bargap=0.05
)
fig.show()